In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [5]:
def convert_dict_from_update_ticks_to_dataframe(tick_data):
    print(tick_data)
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['ratio'] = df_tick_data['last_tick'] - df_tick_data['last_tick'].shift(1)
    df_tick_data = df_tick_data[['option_name', 'strike', 'bid', 'ask', 'last_tick','ratio','stock_price','deal_type_zone','updated_at']]      
    return df_tick_data


In [6]:
updated_ticks = estrategies.update_quotes(just_last_update=False,mode=InformationType.Offline)
ticks_to_process = updated_ticks[0]

options = []
for i in ticks_to_process:
    options.append(i.__dict__)

dataframe_historical_ticks = convert_dict_from_update_ticks_to_dataframe(options)
dataframe_historical_ticks
ratio_statistic_finder = dataframe_historical_ticks[['updated_at','option_name','ratio']]
ratio_statistic_finder = ratio_statistic_finder.set_index('updated_at')
ratio_statistic_finder = ratio_statistic_finder.between_time('10:10', '17:00')
ratio_statistic_finder = ratio_statistic_finder.reset_index()
ratio_statistic_finder = ratio_statistic_finder.pivot(index='updated_at', columns='option_name', values='ratio')
ratio_statistic_finder = ratio_statistic_finder.dropna(axis=1)
ratio_statistic_finder

ratio_statistic_data = {}
for (column, value) in ratio_statistic_finder.iteritems():
    ratio_statistic_data.update({ column: { "Mean": ratio_statistic_finder[column].mean(), "StdDev": ratio_statistic_finder[column].std(), "2xStdDev": (ratio_statistic_finder[column].std()*2)  }}) 

ratio_statistic_dataframe = pd.DataFrame.from_dict(ratio_statistic_data)
ratio_statistic_dataframe = ratio_statistic_dataframe.transpose()

updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Real_Time)
ticks_to_process = updated_ticks[0]
options_updated = []
for i in ticks_to_process:
    options_updated.append(i.__dict__)

df_options_updated = convert_dict_from_update_ticks_to_dataframe(options_updated)
df_options_updated.reset_index()
df_options_updated = df_options_updated.set_index('option_name')
df_options_updated = pd.merge(df_options_updated, ratio_statistic_dataframe, left_index=True, right_index=True)

df_options_updated['above_mean'] = abs(df_options_updated['ratio']) > abs(df_options_updated['Mean'])
df_options_updated['above_2x_std_dev'] = abs(df_options_updated['ratio']) > abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated['a2xStdDev_Price'] = abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated = df_options_updated.drop(columns=['updated_at','StdDev','2xStdDev'])
df_options_updated[df_options_updated['above_mean'] == True]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyError: "['timestamp_option' 'days_to_due_date'] not found in axis"

In [ ]:
df_options_updated